In [17]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, MaxAbsScaler, RobustScaler

from GraphAPI import GraphCreator
from graph_helpers import *
from evaluations import *

%aimport GraphAPI
%aimport graph_helpers
%aimport evaluations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [2]:
gc = GraphCreator("Decision tree", include_see_also=False)
print("Number of Links to Search:", len(gc.next_links), "\n\n")
print(list(gc.primary_nodes.keys()), "\n\n")
print(gc.see_also_articles)

Number of Links to Search: 257 


['Operations research', 'Algorithm', 'Decision analysis', 'Probability', 'Causal model', 'Tree (graph theory)', 'Utility', 'Decision tree learning', 'Decision support system', 'Goal'] 


['Behavior tree (artificial intelligence, robotics and control)', 'Boosting (machine learning)', 'Decision cycle', 'Decision list', 'Decision table', 'Decision tree model', 'Design rationale', 'DRAKON', 'Markov chain', 'Random forest', 'Odds algorithm', 'Topological combinatorics', 'Truth table']


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [3]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect nodes. 

In [4]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [5]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,Hysteria Project 2,Hysteria Project,7
1,Hysteria Project,Hysteria Project 2,7
2,Visualization (graphics),Visual analytics,5
3,Glossary of artificial intelligence,Deep learning,5
4,Visual analytics,Visualization (graphics),5


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [45]:
features_df = gc.get_features_df(rank=False)

In [46]:
features_df.sort_values("degree", ascending=False)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link
0,Logic,3493,0,2794,699,0.005084,9.539776e-02,0.028874,0.214910,2.0,1.0,0.003726,0
1,Algorithm,3381,0,2993,388,0.011272,1.692735e-01,0.022889,0.062707,1.0,2.0,0.008291,1
2,Index of philosophy articles (D–H),3034,0,13,3021,0.002162,3.443527e-04,0.000120,0.006088,3.0,1.0,0.005525,0
3,Time,2896,0,1970,926,0.003509,5.486098e-02,0.019580,0.330584,1.0,3.0,0.000468,0
4,Dungeon Master,2676,0,2342,334,0.000384,7.509537e-06,0.037381,0.178381,5.0,1.0,0.000000,0
5,Machine learning,2609,0,2080,529,0.023376,2.865969e-01,0.013025,0.150376,2.0,1.0,0.017647,0
6,Game theory,2555,0,1795,760,0.013363,1.397245e-01,0.015947,0.305014,2.0,1.0,0.006663,0
7,Causality,2279,0,1280,999,0.007485,6.772202e-02,0.012152,0.292838,1.0,2.0,0.002768,0
8,Glossary of artificial intelligence,2022,0,319,1703,0.020329,1.927794e-01,0.002863,0.082189,2.0,1.0,0.027368,0
9,Probability,1870,0,1576,294,0.013374,1.199586e-01,0.012771,0.073666,1.0,2.0,0.010423,1


# Scaling Features

We can easily scale our each of our features through the `scale_features_df` method. It will default to `Standard Scaler`, but we can specify alternate scalers in the `scaler` parameter.  

In [27]:
gc.scale_features_df(scaler=MinMaxScaler, copy=False)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link
0,Logic,1.000000,0.0,0.933489,0.231126,0.005084,0.332864,0.772384,0.650092,0.250,0.25,0.003726,0.0
1,Algorithm,0.967927,0.0,1.000000,0.128146,0.011272,0.590633,0.612230,0.189687,0.125,0.50,0.008291,1.0
2,Index of philosophy articles (D–H),0.868557,0.0,0.004011,1.000000,0.002162,0.001202,0.002996,0.018416,0.375,0.25,0.005525,0.0
3,Time,0.829038,0.0,0.658088,0.306291,0.003509,0.191422,0.523689,1.000000,0.125,0.75,0.000468,0.0
4,Dungeon Master,0.766037,0.0,0.782420,0.110265,0.000384,0.000026,1.000000,0.539595,0.625,0.25,0.000000,0.0
5,Machine learning,0.746850,0.0,0.694853,0.174834,0.023376,1.000000,0.348297,0.454880,0.250,0.25,0.017647,0.0
6,Game theory,0.731386,0.0,0.599599,0.251325,0.013363,0.487530,0.426485,0.922652,0.250,0.25,0.006663,0.0
7,Causality,0.652348,0.0,0.427473,0.330464,0.007485,0.236297,0.324953,0.885820,0.125,0.50,0.002768,0.0
8,Glossary of artificial intelligence,0.578751,0.0,0.106283,0.563576,0.020329,0.672650,0.076406,0.248619,0.250,0.25,0.027368,0.0
9,Probability,0.535223,0.0,0.526404,0.097020,0.013374,0.418562,0.341508,0.222836,0.125,0.50,0.010423,1.0


# Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [30]:
gc.rank_similarity()
gc.scale_features_df(scaler=MinMaxScaler, copy=False)
gc.features_df.sort_values(["similarity_rank"], ascending=False).reset_index().drop("index", axis=1)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link,similarity_rank
0,Decision tree,0.072738,1.0,0.056150,0.028146,1.000000,3.089506e-01,0.027688,0.062615,0.000,0.00,1.000000,0.0,1.000000
1,Decision analysis,0.033792,1.0,0.023061,0.015894,0.065359,4.199444e-02,0.010892,0.025783,0.250,0.50,0.039130,1.0,0.824598
2,"Behavior tree (artificial intelligence, roboti...",0.006873,1.0,0.001671,0.005960,0.016807,7.364544e-02,0.001581,0.007366,0.250,0.50,0.017442,0.0,0.642106
3,Decision tree learning,0.093643,1.0,0.057152,0.051325,0.067442,4.410999e-01,0.023721,0.165746,0.250,0.50,0.052469,1.0,0.589822
4,Influence diagram,0.015178,1.0,0.006350,0.010927,0.063492,6.987752e-02,0.004659,0.012891,0.250,0.50,0.038462,0.0,0.579485
5,Decision tree model,0.010882,1.0,0.004345,0.007947,0.020000,2.950304e-02,0.003464,0.009208,0.250,0.50,0.022346,0.0,0.575885
6,Grafting (decision trees),0.001145,1.0,0.000334,0.000662,0.008929,1.021027e-02,0.000193,0.000000,0.375,0.50,0.005882,0.0,0.575621
7,Information gain in decision trees,0.015178,1.0,0.003342,0.013907,0.042802,6.665263e-02,0.001761,0.012891,0.250,0.50,0.028571,0.0,0.565984
8,Ahoona,0.002291,1.0,0.000000,0.002318,0.004386,2.828715e-04,0.000305,0.001842,0.375,0.50,0.000000,0.0,0.556675
9,Decision table,0.014891,1.0,0.010027,0.006954,0.034884,1.831377e-02,0.005489,0.012891,0.250,0.50,0.020408,0.0,0.542115


In [47]:
gc.features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42046 entries, 0 to 42045
Data columns (total 13 columns):
node                                 42046 non-null object
degree                               42046 non-null int64
category_matches_with_source         42046 non-null int64
in_edges                             42046 non-null int64
out_edges                            42046 non-null int64
shared_neighbors_with_entry_score    42046 non-null float64
centrality                           42046 non-null float64
page_rank                            42046 non-null float64
adjusted_reciprocity                 42046 non-null float64
shortest_path_length_from_entry      20916 non-null float64
shortest_path_length_to_entry        28671 non-null float64
jaccard_similarity                   42046 non-null float64
primary_link                         42046 non-null int64
dtypes: float64(7), int64(5), object(1)
memory usage: 4.5+ MB


In [43]:
edges = []
for node in gc.graph.nodes:
    node_in_edge = len(gc.graph.in_edges(node))
    node_out_edges = len(gc.graph.out_edges(node))
    edges.append({"node": node, "in_edges": node_in_edge, "out_edges": node_out_edges})
    
df = pd.DataFrame(edges)
df[df.node == "Karyolysis"]

,in_edges,node,out_edges
35666,1,Karyolysis,0


___
# Validation

Here, we _validate_ our results. For many articles, we already have some user defined links that are highly related to the present article. These are found in the **See Also** section of several Wikipedia articles (some pages do not have them). These are not ordered in importance in any meaningful way, and there are no rating scores.

The intuition in this validation is as follows: 

> _Given that we know some articles are highly related from user input, if the recommendations provided by this system are valid, we would expect to see those **See Also** links ranked relatively high on our list._ 

_Note: This validation is not meant as **confirmation** or **evaluation** of the results. It only provides us one way of telling if the results we are seeing are reasonably valid. It is important to note that we cannot compare these results across two different articles, as those would be two entirely different network structures, likely with different human labeled links._  


In [9]:
evaluate_metrics(gc.features_df, 
                 on=["similarity_rank", "centrality", "adjusted_reciprocity", "page_rank", "shortest_path_length_from_entry", "jaccard_similarity"], 
                 targets=gc.see_also_articles)

Metric Score,score,max score possible,difference,total targets,% targets in top 1%,% targets in top 5%,% targets in top 10%,% targets in top 20%
similarity_rank,0.979871,0.999921,0.020050,13.0,0.923077,1.000000,1.000000,1.000000
centrality,0.952690,0.999921,0.047231,13.0,0.230769,1.000000,1.000000,1.000000
adjusted_reciprocity,0.285551,0.999921,0.714370,13.0,0.461538,0.769231,0.846154,0.846154
page_rank,0.955745,0.999921,0.044176,13.0,0.538462,1.000000,1.000000,1.000000
shortest_path_length_from_entry,0.727489,0.999921,0.272432,13.0,0.000000,0.000000,0.000000,0.000000
jaccard_similarity,0.496982,0.999921,0.502939,13.0,0.230769,0.461538,0.615385,0.615385


The chart generated above compares different ranking metrics (left index) for a given article. The most important column, `score`, provides a fast way for us to compare these different metrics. 

For example, if we see a _score_ of 0.98 for a given ranking metric, The following statement would be true:

> All of the human labeled **See Also** links are present within the top 98% of our recommendations. 

Since the human labeled links comprise a range, it is not possible to get a score of 100%. The `max score possible` column indicates the score that would be achieved if all the human labeled _See Also_ links appeared at the top of our recommendations without any other links intervening.   

The `difference` column is an alternative way of looking at the score. If we had a 0.02 in this column, we could say:

> All the human labeled **See Also** links are contained within the top 2% of our recommendations. 

`Total targets` is the number of human labeled _See Also_ links. 

Because it is possible that different metrics could have similar scores, we want a way to break down the dispersion of the known related links to see if one metric does perform better than another. The trailing four columns provide us with a course way of measuring this dispersion. 

Each of these columns indicates the percentage of human labeled _See Also_ links captured within a given percentage of the top of our recommendations. For example, if we see a 0.92 in the `% targets in the top 1%` columns, we could say:

> 92% of the human labeled **See Also** links appear in the top 1% of our recommendations. 

The value of these columns is a follows - If two ranking metrics have similar scores, we _might_ consider the better performing one to be the one in which the majority of the human labeled links are higher in our recommendation list. 

___

In [10]:
df = gc.features_df.sort_values("similarity_rank", ascending=False).reset_index().drop("index", axis=1)
see_also_indeces = []
for node in gc.see_also_articles:
    article = df[df.node == node]
    
    see_also_indeces.append((node, article.index[0]))
    
see_also_indeces

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
len(gc.graph.nodes)